In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
#install evaluate
!pip install evaluate

In [10]:
#bertscore
#!pip install bert_score

In [21]:
from transformers import AutoModelForCausalLM, GPT2Tokenizer, AutoTokenizer
import torch
import numpy as np
from evaluate import load
bertscore = load("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")
print(results)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'precision': [1.0000001192092896, 0.9999999403953552], 'recall': [1.0000001192092896, 0.9999999403953552], 'f1': [1.0000001192092896, 0.9999999403953552], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.31.0)'}


In [48]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
#!pip install -r requirements.txt

In [23]:
def generate(context, model, tokenizer):
      # create prompt
  prompt = context + "agent: "

  # tokenize input (string -> tensor)
  input_ids = tokenizer(prompt, return_tensors='pt').input_ids

  # call model generate (https://huggingface.co/docs/transformers/v4.34.0/en/main_classes/text_generation#transformers.GenerationMixin.generate)
  output_ids = model.generate(input_ids, max_length=800, eos_token_id=
                              tokenizer.encode("customer: "))
  
  # decode output (tensor -> string)
  pred_response = tokenizer.batch_decode(output_ids, skip_special_tokens = True)
  
  output_string=pred_response[0]
  prefix = context + " agent: "
  suffix = "customer: "
  sliced_output = output_string.removeprefix(prefix)  
  
  return sliced_output

In [20]:
#distilGPT2 
from evaluate import load
bertscore = load("bertscore")

import pandas as pd

model = AutoModelForCausalLM.from_pretrained( "../finetuning/distilgpt2-finetuned-wikitext2/")
tokenizer = AutoTokenizer.from_pretrined("gpt2")

test_data = load("evaluate_final_data.xlsx")
context_response_pairs = subsample(test_data, n=1000) 

output_df = pd.DataFrame(columns=['context','true_agent_response','predicted_response'])
# loop over the test set and collect pred, true responses into a list
all_pred_responses, all_true_responses = [], []

for context, true_response in context_response_pairs:
    pred_response = generate(context, model, tokenizer)
    all_pred_responses.append(pred_response)
    all_true_responses.append(true_response)
    results = bertscore.compute(all_pred_responses, all_true_responses))
    output_df = pd.concat([output_df, pd.DataFrame([])], ignore_index= True)
  
output_df.to_csv("../data/output2.csv")
  

In [50]:
# Set up model, tokenizer for evaluation
from evaluate import load
bertscore = load("bertscore")

model = AutoModelForCausalLM.from_pretrained( "../finetuning/distilgpt2-finetuned-wikitext2/")
#model = AutoModelForCausalLM.from_pretrained( "../finetuning/gpt2-finetuned-wikitext2/")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Looping through all rows in test_data
import pandas as pd

df = pd.read_excel("../data/evaluate_final_data.xlsx")

df_subsample = df.sample(n=1000, random_state=1) 
output_df = pd.DataFrame(columns=["context", "true_agent_response", "predicted_response", 'BERT_precision', 'BERT_recall', "BERT_f1"])
all_pred_responses, all_true_responses = [], []

for index, row in df_subsample.iterrows():
    pred_response = generate(row["context"], model, tokenizer)
    all_pred_responses.append(pred_response)
    all_true_responses.append(row["true_agent_response"]) 
    similarity_score = bertscore.compute(predictions=[pred_response], references=[row["true_agent_response"]], lang="en", rescale_with_baseline = True)
    new_row = {"context": row["context"], "true_agent_response":row["true_agent_response"], "predicted_response": pred_response, 'BERT_precision':similarity_score["precision"][0], "BERT_recall": similarity_score["recall"][0], "BERT_f1": similarity_score["f1"][0]}
    output_df = pd.concat([output_df, pd.DataFrame([new_row])], ignore_index=True)

output_df.to_csv("../data/output_D2.csv")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:23144 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `

In [51]:
output_df['BERT_f1'] = np.where(output_df['BERT_f1'] < 0, 0, output_df['BERT_f1'])
output_df[['BERT_f1']]

0      0.035217
1      0.106313
2      0.000000
3      0.000000
4      0.000000
         ...   
995    0.000000
996    0.011784
997    0.000000
998    0.024421
999    0.000000
Name: BERT_f1, Length: 1000, dtype: float64

In [52]:
np.mean(output_df['BERT_f1'])

0.024117715197149663

In [53]:
output_df.columns

Index(['context', 'true_agent_response', 'predicted_response',
       'BERT_precision', 'BERT_recall', 'BERT_f1'],
      dtype='object')

In [54]:
results

{'precision': [1.0000001192092896, 0.9999999403953552],
 'recall': [1.0000001192092896, 0.9999999403953552],
 'f1': [1.0000001192092896, 0.9999999403953552],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.31.0)'}

``` python 
model_generate(input_ids, max_length=50, eos_token_id=tokenizer.encode("Customer:")[0])
output_ids=outputs[:, input_ids.shape[-1]:]
# may need to flatten all_pred_responses list before passing to bertscore()